In [1]:
# !pip install 'qiskit[visualization]'
# !pip install qiskit-ibm-runtime
# !pip install qiskit_algorithms
# !pip install qiskit-aer
# !pip install pyscf

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from pyscf import ao2mo, fci, gto, mcscf, scf, tools
from qiskit.quantum_info import SparsePauliOp


def cholesky(V, eps):
    # see https://arxiv.org/pdf/1711.02242.pdf section B2
    # see https://arxiv.org/abs/1808.02625
    # see https://arxiv.org/abs/2104.08957
    no = V.shape[0]
    chmax, ng = 20 * no, 0
    W = V.reshape(no**2, no**2)
    L = np.zeros((no**2, chmax))
    Dmax = np.diagonal(W).copy()
    nu_max = np.argmax(Dmax)
    vmax = Dmax[nu_max]
    while vmax > eps:
        L[:, ng] = W[:, nu_max]
        if ng > 0:
            L[:, ng] -= np.dot(L[:, 0:ng], (L.T)[0:ng, nu_max])
        L[:, ng] /= np.sqrt(vmax)
        Dmax[: no**2] -= L[: no**2, ng] ** 2
        ng += 1
        nu_max = np.argmax(Dmax)
        vmax = Dmax[nu_max]
    L = L[:, :ng].reshape((no, no, ng))
    print(
        "accuracy of Cholesky decomposition ",
        np.abs(np.einsum("prg,qsg->prqs", L, L) - V).max(),
    )
    return L, ng


def identity(n):
    return SparsePauliOp.from_list([("I" * n, 1)])


def creators_destructors(n, mapping="jordan_wigner"):
    c_list = []
    if mapping == "jordan_wigner":
        for p in range(n):
            if p == 0:
                l, r = "I" * (n - 1), ""
            elif p == n - 1:
                l, r = "", "Z" * (n - 1)
            else:
                l, r = "I" * (n - p - 1), "Z" * p
            cp = SparsePauliOp.from_list([(l + "X" + r, 0.5), (l + "Y" + r, -0.5j)])
            c_list.append(cp)
    else:
        raise ValueError("Unsupported mapping.")
    d_list = [cp.adjoint() for cp in c_list]
    return c_list, d_list


def build_hamiltonian(ecore: float, h1e: np.ndarray, h2e: np.ndarray) -> SparsePauliOp:
    ncas, _ = h1e.shape

    C, D = creators_destructors(2 * ncas, mapping="jordan_wigner")
    Exc = []
    for p in range(ncas):
        Excp = [C[p] @ D[p] + C[ncas + p] @ D[ncas + p]]
        for r in range(p + 1, ncas):
            Excp.append(
                C[p] @ D[r]
                + C[ncas + p] @ D[ncas + r]
                + C[r] @ D[p]
                + C[ncas + r] @ D[ncas + p]
            )
        Exc.append(Excp)

    # low-rank decomposition of the Hamiltonian
    Lop, ng = cholesky(h2e, 1e-6)
    t1e = h1e - 0.5 * np.einsum("pxxr->pr", h2e)

    H = ecore * identity(2 * ncas)
    # one-body term
    for p in range(ncas):
        for r in range(p, ncas):
            H += t1e[p, r] * Exc[p][r - p]
    # two-body term
    for g in range(ng):
        Lg = 0 * identity(2 * ncas)
        for p in range(ncas):
            for r in range(p, ncas):
                Lg += Lop[p, r, g] * Exc[p][r - p]
        H += 0.5 * Lg @ Lg

    return H.chop().simplify()

__Example molecules__

In [3]:
# LiH

distance = 0.735
mol = gto.Mole()
mol.build(
    verbose=0,
    atom=[["Li", (0, 0, 0)], ["H", (0, 0, distance)]],
    basis="sto-6g",
    spin=0,
    charge=0,
    symmetry="Coov",
)
cas_space_symmetry = {"A1": 3, "E1x": 1, "E1y": 1}

# -----------Building the fermionic Hamiltonian---------------------------------------------------------------------------

mf = scf.RHF(mol)
E1 = mf.kernel()
mx = mcscf.CASCI(mf, ncas=5, nelecas=(1, 1))
mo = mcscf.sort_mo_by_irrep(mx, mf.mo_coeff, cas_space_symmetry)

E2 = mx.kernel(mo)[:2]
h1e, ecore = mx.get_h1eff()
h2e = ao2mo.restore(1, mx.get_h2eff(), mx.ncas)

# ------------Building the QC Hamiltonian----------------------------------------------------------------------------------

H = build_hamiltonian(ecore, h1e, h2e)
print(H)

accuracy of Cholesky decomposition  2.220446049250313e-16
SparsePauliOp(['IIIIIIIIII', 'IIIIIIIIIZ', 'IIIIZIIIII', 'IIIIIIIIYY', 'IIIIIIIIXX', 'IIIYYIIIII', 'IIIXXIIIII', 'IIIIIYZZZY', 'IIIIIXZZZX', 'YZZZYIIIII', 'XZZZXIIIII', 'IIIIIIIIZI', 'IIIZIIIIII', 'IIIIIYZZYI', 'IIIIIXZZXI', 'YZZYIIIIII', 'XZZXIIIIII', 'IIIIIIIZII', 'IIZIIIIIII', 'IIIIIIZIII', 'IZIIIIIIII', 'IIIIIZIIII', 'ZIIIIIIIII', 'IIIIZIIIIZ', 'IIIYYIIIIZ', 'IIIXXIIIIZ', 'YZZZYIIIIZ', 'XZZZXIIIIZ', 'IIIIIIIIZZ', 'IIIZIIIIIZ', 'IIIIIYZZYZ', 'IIIIIXZZXZ', 'YZZYIIIIIZ', 'XZZXIIIIIZ', 'IIIIIIIZIZ', 'IIZIIIIIIZ', 'IIIIIIZIIZ', 'IZIIIIIIIZ', 'IIIIIZIIIZ', 'ZIIIIIIIIZ', 'IIIIZIIIYY', 'IIIIZIIIXX', 'IIIIZYZZZY', 'IIIIZXZZZX', 'IIIIZIIIZI', 'IIIZZIIIII', 'IIIIZYZZYI', 'IIIIZXZZXI', 'YZZYZIIIII', 'XZZXZIIIII', 'IIIIZIIZII', 'IIZIZIIIII', 'IIIIZIZIII', 'IZIIZIIIII', 'IIIIZZIIII', 'ZIIIZIIIII', 'IIIIIYZZIY', 'IIIYYIIIYY', 'IIIXXIIIYY', 'YZZZYIIIYY', 'XZZZXIIIYY', 'IIIZIIIIYY', 'YZZYIIIIYY', 'XZZXIIIIYY', 'IIIIIIIZYY', 'IIZIIIIIYY', 'II

In [7]:
# H20

# %% ------------- PySCF part: define a molecule and carve an active space

distance = 0.957         # known bond length
thetas_in_deg = 104.478  # bond angles.

H1_x = distance
H2_x = distance * np.cos(np.pi / 180 * thetas_in_deg)
H2_y = distance * np.sin(np.pi / 180 * thetas_in_deg)

mol = gto.Mole()
mol.build(
    verbose=0,
    atom=[
        ["O", (0, 0, 0)],
        ["H", (H1_x, 0, 0)],
        ["H", (H2_x, H2_y, 0)],
    ],
    basis="sto-6g",
    spin=0,
    charge=0,
    symmetry="C2v",
)
cas_space_symmetry = {"A1": 3, "B1": 1, "B2": 2}

# -----------Building the fermionic Hamiltonian---------------------------------------------------------------------------

mf = scf.RHF(mol)
E1 = mf.kernel()
mx = mcscf.CASCI(mf, ncas=6, nelecas=(4, 4))
mo = mcscf.sort_mo_by_irrep(mx, mf.mo_coeff, cas_space_symmetry)


E2 = mx.kernel(mo)[:2]
h1e, ecore = mx.get_h1eff()
h2e = ao2mo.restore(1, mx.get_h2eff(), mx.ncas)

# ------------Building the QC Hamiltonian----------------------------------------------------------------------------------

H = build_hamiltonian(ecore, h1e, h2e)
print(f"Ground state energy by RHF calculations = {E1}")
print(f"Ground state energy by CASCI calculation = {E2}")
print(f"Core energby (frozen electrons + nuclear nuclear repulsion) = {ecore}")
print(f"One-electron integral = {np.round(h1e, decimals = 3)}")
print(f"two electron inegral = {np.round(h2e, decimals =5)}")
print(f"Shape of one electron integral = {np.shape(h1e)}")
print(f"Shape of two electron integral = {np.shape(h2e)}")
print(H)

accuracy of Cholesky decomposition  2.220446049250313e-16
Ground state energy by RHF calculations = -75.6786634953089
Ground state energy by CASCI calculation = (np.float64(-75.72857205617773), np.float64(-23.640289183423363))
Core energby (frozen electrons + nuclear nuclear repulsion) = -52.08828287275436
One-electron integral = [[-5.744 -0.     0.2    0.     0.811 -0.   ]
 [-0.    -4.791 -0.     0.     0.     1.004]
 [ 0.2   -0.    -5.033  0.     0.638  0.   ]
 [ 0.     0.     0.    -5.268  0.     0.   ]
 [ 0.811  0.     0.638  0.    -3.785 -0.   ]
 [-0.     1.004  0.     0.    -0.    -3.901]]
two electron inegral = [[[[ 0.73103  0.       0.00322 -0.      -0.14506 -0.     ]
   [ 0.       0.64683  0.      -0.      -0.      -0.13978]
   [ 0.00322  0.       0.67847 -0.      -0.09563 -0.     ]
   [-0.      -0.      -0.       0.74992 -0.      -0.     ]
   [-0.14506 -0.      -0.09563 -0.       0.61656 -0.     ]
   [-0.      -0.13978 -0.      -0.      -0.       0.62513]]

  [[ 0.       0.14

In [5]:
# ethene

# %% ------------- PySCF part: define a molecule and carve an active space

distance = 1.33     #C-C bond length in ethene
a = distance / 2
b = a + 0.5626
c = 0.9289

mol = gto.Mole()
mol.build(
    verbose=0,
    atom=[
        ["C", (0, 0, a)],
        ["C", (0, 0, -a)],
        ["H", (0, c, b)],
        ["H", (0, -c, b)],
        ["H", (0, c, -b)],
        ["H", (0, -c, -b)],
    ],
    basis="sto-6g",
    spin=0,
    charge=0,
    symmetry="d2h",
)
active_space = range(mol.nelectron // 2 - 2, mol.nelectron // 2 + 2)

# -----------Building the fermionic Hamiltonian---------------------------------------------------------------------------

mf = scf.RHF(mol)
E1 = mf.kernel()
mx = mcscf.CASCI(mf, ncas=4, nelecas=(2, 2))
mo = mx.sort_mo(active_space, base=0)

E2 = mx.kernel(mo)[:2]
h1e, ecore = mx.get_h1eff()
h2e = ao2mo.restore(1, mx.get_h2eff(), mx.ncas)

# ------------Building the QC Hamiltonian----------------------------------------------------------------------------------

H = build_hamiltonian(ecore, h1e, h2e)
print(H)

accuracy of Cholesky decomposition  1.6653345369377348e-16
SparsePauliOp(['IIIIIIII', 'IIIIIIIZ', 'IIIZIIII', 'IIIIIIZI', 'IIZIIIII', 'IIIIIZII', 'IZIIIIII', 'IIIIZIII', 'ZIIIIIII', 'IIIZIIIZ', 'IIIIIIZZ', 'IIZIIIIZ', 'IIIIIZIZ', 'IZIIIIIZ', 'IIIIZIIZ', 'ZIIIIIIZ', 'IIIZIIZI', 'IIZZIIII', 'IIIZIZII', 'IZIZIIII', 'IIIZZIII', 'ZIIZIIII', 'IIZIIIZI', 'IIIIIZZI', 'IZIIIIZI', 'IIIIZIZI', 'ZIIIIIZI', 'IIZIIZII', 'IZZIIIII', 'IIZIZIII', 'ZIZIIIII', 'IZIIIZII', 'IIIIZZII', 'ZIIIIZII', 'IZIIZIII', 'ZZIIIIII', 'ZIIIZIII', 'IIIIXXYY', 'YZZYYZZY', 'XZZXYZZY', 'IIIIYXXY', 'IIIIYYYY', 'IYYIYZZY', 'IXXIYZZY', 'YZZYXZZX', 'XZZXXZZX', 'IIIIXXXX', 'IIIIXYYX', 'IYYIXZZX', 'IXXIXZZX', 'IIIIYYXX', 'XXYYIIII', 'YZZYIYYI', 'YZZYIXXI', 'YXXYIIII', 'YYYYIIII', 'XZZXIYYI', 'XZZXIXXI', 'XXXXIIII', 'XYYXIIII', 'YYXXIIII', 'IYYIIYYI', 'IXXIIYYI', 'IYYIIXXI', 'IXXIIXXI', 'IYZYIYZY', 'IXZXIYZY', 'YZYIIYZY', 'XZXIIYZY', 'IYZYIXZX', 'IXZXIXZX', 'YZYIIXZX', 'XZXIIXZX', 'IYZYYZYI', 'IYZYXZXI', 'IXZXYZYI', 'IXZXXZXI', 'Y

In [6]:
# H2 with explicit gerade & ungerade orbitals

# %% ------------- PySCF part: define a molecule and carve an active space
distance = 0.735
a = distance / 2


mol = gto.Mole()
mol.build(
    verbose=0,
    atom=[
        ["H", (0, 0, -a)],
        ["H", (0, 0, a)],
    ],
    basis="sto-6g",
    spin=0,
    charge=0,
    symmetry="Dooh",
)
cas_space_symmetry = {"A1g": 1, "A1u": 1}

# -----------Building the fermionic Hamiltonian---------------------------------------------------------------------------

mf = scf.RHF(mol)
E1 = mf.kernel()
mx = mcscf.CASCI(mf, ncas=2, nelecas=(1, 1))
mo = mcscf.sort_mo_by_irrep(mx, mf.mo_coeff, cas_space_symmetry)

E2 = mx.kernel(mo)[:2]
h1e, ecore = mx.get_h1eff()
h2e = ao2mo.restore(1, mx.get_h2eff(), mx.ncas)

# ------------Building the QC Hamiltonian----------------------------------------------------------------------------------

H = build_hamiltonian(ecore, h1e, h2e)
print(H)

accuracy of Cholesky decomposition  1.8768594323382656e-16
SparsePauliOp(['IIII', 'IIIZ', 'IZII', 'IIZI', 'ZIII', 'IZIZ', 'IIZZ', 'ZIIZ', 'IZZI', 'ZZII', 'ZIZI', 'YYYY', 'XXYY', 'YYXX', 'XXXX'],
              coeffs=[-0.09820182+0.j,  0.1740751 +0.j,  0.1740751 +0.j, -0.2242933 +0.j,
 -0.2242933 +0.j,  0.16891402+0.j,  0.1210099 +0.j,  0.16631441+0.j,
  0.16631441+0.j,  0.1210099 +0.j,  0.17504456+0.j,  0.04530451+0.j,
  0.04530451+0.j,  0.04530451+0.j,  0.04530451+0.j])


In [7]:
# H2 numbered active space orbitals

# %% ------------- PySCF part: define a molecule and carve an active space
distance = 0.735
a = distance / 2


mol = gto.Mole()
mol.build(
    verbose=0,
    atom=[
        ["H", (0, 0, -a)],
        ["H", (0, 0, a)],
    ],
    basis="sto-6g",
    spin=0,
    charge=0,
    symmetry="Dooh",
)
active_space = range(mol.nelectron // 2-1, mol.nelectron // 2 + 1) # Note changes here vs. above

# -----------Building the fermionic Hamiltonian---------------------------------------------------------------------------

mf = scf.RHF(mol)
E1 = mf.kernel()
mx = mcscf.CASCI(mf, ncas=2, nelecas=(1, 1))
mo = mx.sort_mo(active_space, base=0)           #Also syntax changes here vs. above

E2 = mx.kernel(mo)[:2]
h1e, ecore = mx.get_h1eff()
h2e = ao2mo.restore(1, mx.get_h2eff(), mx.ncas)

# ------------Building the QC Hamiltonian----------------------------------------------------------------------------------

H = build_hamiltonian(ecore, h1e, h2e)
print(H)

accuracy of Cholesky decomposition  1.8768594323382656e-16
SparsePauliOp(['IIII', 'IIIZ', 'IZII', 'IIZI', 'ZIII', 'IZIZ', 'IIZZ', 'ZIIZ', 'IZZI', 'ZZII', 'ZIZI', 'YYYY', 'XXYY', 'YYXX', 'XXXX'],
              coeffs=[-0.09820182+0.j,  0.1740751 +0.j,  0.1740751 +0.j, -0.2242933 +0.j,
 -0.2242933 +0.j,  0.16891402+0.j,  0.1210099 +0.j,  0.16631441+0.j,
  0.16631441+0.j,  0.1210099 +0.j,  0.17504456+0.j,  0.04530451+0.j,
  0.04530451+0.j,  0.04530451+0.j,  0.04530451+0.j])


In [8]:
# O2 numbered active space orbitals

    # %% ------------- PySCF part: define a molecule and carve an active space

distance = 1.16
a = distance / 2

mol = gto.Mole()
mol.build(
    verbose=0,
    atom=[
        ["O", (0, 0, a)],
        ["O", (0, 0, -a)],
    ],
    basis="sto-6g",
    spin=0,
    charge=0,
    symmetry="d2h",
)

active_space = range(mol.nelectron // 2 - 2, mol.nelectron // 2 + 2)

# -----------Building the fermionic Hamiltonian---------------------------------------------------------------------------

mf = scf.RHF(mol)
E1 = mf.kernel()
mx = mcscf.CASCI(mf, ncas=4, nelecas=(2, 2))
mo = mx.sort_mo(active_space, base=0)

E2 = mx.kernel(mo)[:2]
h1e, ecore = mx.get_h1eff()
h2e = ao2mo.restore(1, mx.get_h2eff(), mx.ncas)

# ------------Building the QC Hamiltonian----------------------------------------------------------------------------------

H = build_hamiltonian(ecore, h1e, h2e)
print(H)

accuracy of Cholesky decomposition  1.1102230246251565e-16
SparsePauliOp(['IIIIIIII', 'IIIIIIIZ', 'IIIZIIII', 'IIIIIIZI', 'IIZIIIII', 'IIIIIZII', 'IZIIIIII', 'IIIIZIII', 'ZIIIIIII', 'IIIZIIIZ', 'IIIIIIZZ', 'IIZIIIIZ', 'IIIIIZIZ', 'IZIIIIIZ', 'IIIIZIIZ', 'ZIIIIIIZ', 'IIIZIIZI', 'IIZZIIII', 'IIIZIZII', 'IZIZIIII', 'IIIZZIII', 'ZIIZIIII', 'IIZIIIZI', 'IIIIIZZI', 'IZIIIIZI', 'IIIIZIZI', 'ZIIIIIZI', 'IIZIIZII', 'IZZIIIII', 'IIZIZIII', 'ZIZIIIII', 'IZIIIZII', 'IIIIZZII', 'ZIIIIZII', 'IZIIZIII', 'ZZIIIIII', 'ZIIIZIII', 'IYZYIYZY', 'IXZXIYZY', 'IYZYIXZX', 'IXZXIXZX', 'YZYIYZYI', 'XZXIYZYI', 'YZYIXZXI', 'XZXIXZXI', 'YYIIYYII', 'XXIIYYII', 'YYIIXXII', 'XXIIXXII', 'YZZYYZZY', 'XZZXYZZY', 'YZZYXZZX', 'XZZXXZZX', 'IYYIIYYI', 'IXXIIYYI', 'IYYIIXXI', 'IXXIIXXI', 'IIYYIIYY', 'IIXXIIYY', 'IIYYIIXX', 'IIXXIIXX'],
              coeffs=[-1.47572001e+02+0.j,  1.45553793e-01+0.j,  1.45553793e-01+0.j,
  1.77702110e-03+0.j,  1.77702110e-03+0.j, -2.75967421e-02+0.j,
 -2.75967421e-02+0.j, -2.74813383e-01+0.j, -